In [145]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, ttest_ind, wilcoxon, mannwhitneyu, chi2_contingency, norm, pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [99]:
def fill_missing_date(start_date, end_date):
    df = pd.read_csv('data/daily_performance.csv')
    print(f'Number of rows BEFORE: {len(df)}')
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    all_dates = pd.date_range(pd.to_datetime(start_date), pd.to_datetime(end_date), freq='D')
    df = df.reindex(all_dates, fill_value=0)
    df.index.name = 'date'
    df = df.reset_index()
    df['no_click'] = df['impression'] - df['click']
    print(f'Number of rows AFTER: {len(df)}')

    return df

In [101]:
def split_data(df, current_start, current_end, prev_start, prev_end):
    current_df = df[(df['date'] >= pd.to_datetime(current_start)) & (df['date'] <= pd.to_datetime(current_end))]
    previous_df = df[(df['date'] >= pd.to_datetime(prev_start)) & (df['date'] <= pd.to_datetime(prev_end))]

    return current_df, previous_df

In [103]:
def sig_test(current_df, previous_df, metric):
    shapiro_current = 1
    shapiro_prev = 1
    
    if len(current_df) < 30 or len(previous_df) < 30:
        shapiro_current = shapiro(current_df[metric])[1]
        shapiro_prev = shapiro(previous_df[metric])[1]
    
        print(f'Shapiro test p-value for current period: {shapiro_current}')
        print(f'Shapiro test p-value for previous period: {shapiro_prev}\n')

    if shapiro_current <= 0.05 or shapiro_prev <= 0.05:
        if len(current_df) != len(previous_df):
            p = mannwhitneyu(current_df[metric], previous_df[metric])[1]
            test_used = 'Mann-Whitney U test'
        else:
            p = wilcoxon(current_df[metric], previous_df[metric])[1]
            test_used = 'Wilcoxon test'
    else:
        p = ttest_ind(current_df[metric], previous_df[metric])[1]
        test_used = 't-test'
        

    print(f'The p-value of {test_used} is {p}\n')
    print(f'Average {metric} in current period: {current_df[metric].mean()}')
    print(f'Average {metric} in previous period: {previous_df[metric].mean()}\n')
    if p <= 0.05:
        print(f'The {metric} in current period is significantly different from the previous period.')
    else:
        print(f'We CANNOT conclude that the {metric} in current period is significantly different from the previous period.')

In [105]:
def prop_test(current_df, previous_df, metric):
    current_impression, previous_impression = current_df['impression'].sum(), previous_df['impression'].sum()
    current_click, previous_click = current_df['click'].sum(), previous_df['click'].sum()
    current_conversion, previous_conversion = current_df['conversion'].sum(), previous_df['conversion'].sum()
    
    if metric == 'click':
        current_rate = current_click / current_impression
        previous_rate = previous_click / previous_impression
        count = [current_click, previous_click]
        nobs = [current_impression, previous_impression]
        rate = 'CTR'
    elif metric == 'conversion':
        current_rate = current_conversion / current_click
        previous_rate = previous_conversion / previous_click
        count = [current_conversion, previous_conversion]
        nobs = [current_click, previous_click]
        rate = 'Conversion Rate'
        
    z_stat, p_value = sm.stats.proportions_ztest(count, nobs)
    
    print(f'Current {rate}: {current_rate:.4f}')
    print(f'Previous {rate}: {previous_rate:.4f}\n')
    print(f'p-value: {p_value}')
    if p_value <= 0.05:
        print(f'The {rate} in current period is significantly different from the previous period')
    else:
        print(f'We CANNOT conlcude that the {rate} is significantly different')

In [107]:
start_date = '2024-01-01'
end_date = '2024-08-19'

df = fill_missing_date(start_date, end_date)

Number of rows BEFORE: 175
Number of rows AFTER: 232


In [133]:
current_start = '2024-07-26'
current_end = '2024-08-19'
previous_start = '2024-06-01'
previous_end = '2024-07-25'

df_current, df_previous = split_data(df, current_start, current_end, previous_start, previous_end)

In [135]:
sig_test(df_current, df_previous, 'impression')

Shapiro test p-value for current period: 0.0005109914927743375
Shapiro test p-value for previous period: 4.7330315283034e-05

The p-value of Mann-Whitney U test is 0.1134122048369055

Average impression in current period: 229.8
Average impression in previous period: 343.25454545454545

We CANNOT conclude that the impression in current period is significantly different from the previous period.


In [137]:
sig_test(df_current, df_previous, 'click')

Shapiro test p-value for current period: 0.0006692656315863132
Shapiro test p-value for previous period: 1.1597347111091949e-05

The p-value of Mann-Whitney U test is 0.051559309560519656

Average click in current period: 34.08
Average click in previous period: 54.85454545454545

We CANNOT conclude that the click in current period is significantly different from the previous period.


In [139]:
sig_test(df_current, df_previous, 'conversion')

Shapiro test p-value for current period: 6.702615064568818e-05
Shapiro test p-value for previous period: 7.339947741399552e-14

The p-value of Mann-Whitney U test is 2.03814107915213e-05

Average conversion in current period: 2.44
Average conversion in previous period: 0.32727272727272727

The conversion in current period is significantly different from the previous period.


In [141]:
prop_test(df_current, df_previous, 'click')

Current CTR: 0.1483
Previous CTR: 0.1598

p-value: 0.03590148025838621
The CTR in current period is significantly different from the previous period


In [143]:
prop_test(df_current, df_previous, 'conversion')

Current Conversion Rate: 0.0716
Previous Conversion Rate: 0.0060

p-value: 5.6704680997616444e-33
The Conversion Rate in current period is significantly different from the previous period
